In [ ]:
pip install datasets transformers peft torch accelerate evaluate

## Load 100 Samples from the Dolly-15k Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("databricks/databricks-dolly-15k", split="train[:100]")

print(f"Loaded {len(dataset)} samples.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded 100 samples.


In [ ]:
# Print the first sample
print("Sample from Dolly-15k:")
print(dataset[0])

Sample from Dolly-15k:
{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


## Load the FLAN-T5 Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"Loaded FLAN-T5 model: {model_name}")

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Loaded FLAN-T5 model: google/flan-t5-base


## Prepare the Dataset for Fine-Tuning

In [ ]:
from datasets import Dataset

# Function to preprocess samples
def preprocess_function(examples):
    inputs = [f"Instruction: {inst} Context: {ctx}" for inst, ctx in zip(examples['instruction'], examples['context'])]
    targets = examples['response']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split into train and eval
train_dataset = tokenized_dataset.select(range(80))
eval_dataset = tokenized_dataset.select(range(80, 100))

## Apply LoRA PEFT Method

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,
    target_modules=["q", "v"]  # Target attention layers
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

print("LoRA applied to FLAN-T5.")
lora_model.print_trainable_parameters()  # Shows trainable params (should be ~0.1% of total)

LoRA applied to FLAN-T5.
trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


## Apply IA3 PEFT Method

In [ ]:
from peft import IA3Config

# Reload base model for IA3
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# IA3 configuration
ia3_config = IA3Config(
    task_type=TaskType.SEQ_2_SEQ_LM,
    target_modules=["k", "v", "wo"],  # Modules to apply IA3 to
    feedforward_modules=["wo"]
)

# Apply IA3 to the model
ia3_model = get_peft_model(model, ia3_config)

print("IA3 applied to FLAN-T5.")
ia3_model.print_trainable_parameters()  # Even fewer trainable params than LoRA

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


IA3 applied to FLAN-T5.
trainable params: 104,448 || all params: 247,682,304 || trainable%: 0.0422


## Fine-Tune the Model with PEFT `LoRA`

In [ ]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq, Trainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=lora_model,
    label_pad_token_id=-100
)

# Create Trainer for LoRA
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=data_collator
)

# Fine-tune
trainer.train()

Epoch,Training Loss,Validation Loss
1,11.173636,11.257123
2,11.173354,11.255841
3,11.261704,11.253555


TrainOutput(global_step=60, training_loss=11.206020991007486, metrics={'train_runtime': 64.013, 'train_samples_per_second': 3.749, 'train_steps_per_second': 0.937, 'total_flos': 164994062745600.0, 'train_loss': 11.206020991007486, 'epoch': 3.0})

## Fine-Tune the Model with PEFT `IA3`

In [ ]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq, Trainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=ia3_model,
    label_pad_token_id=-100
)

# Create Trainer for IA3
trainer = Trainer(
    model=ia3_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=data_collator
)

# Fine-tune
trainer.train()

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Epoch,Training Loss,Validation Loss
1,11.153227,11.257408
2,11.155965,11.257048
3,11.277359,11.256445


TrainOutput(global_step=60, training_loss=11.199097696940104, metrics={'train_runtime': 63.3656, 'train_samples_per_second': 3.788, 'train_steps_per_second': 0.947, 'total_flos': 164418772008960.0, 'train_loss': 11.199097696940104, 'epoch': 3.0})

## Evaluate LoRA Model

In [ ]:
lora_eval_results = trainer.evaluate(eval_dataset)
print("LoRA Model Evaluation Results:")
print(lora_eval_results)

LoRA Model Evaluation Results:
{'eval_loss': 11.256444931030273, 'eval_runtime': 2.0741, 'eval_samples_per_second': 9.643, 'eval_steps_per_second': 2.411, 'epoch': 3.0}


## Evaluate IA3 Model

In [ ]:
ia3_eval_results = trainer.evaluate(eval_dataset)
print("IA3 Model Evaluation Results:")
print(ia3_eval_results)

IA3 Model Evaluation Results:
{'eval_loss': 11.256444931030273, 'eval_runtime': 2.0348, 'eval_samples_per_second': 9.829, 'eval_steps_per_second': 2.457, 'epoch': 3.0}


## Compare LoRA and IA3 Results


In [ ]:
print(f"LoRA eval_loss: {lora_eval_results['eval_loss']}")
print(f"IA3 eval_loss: {ia3_eval_results['eval_loss']}")

print("\n--- Detailed Comparison ---")
print("LoRA Evaluation Results:")
for key, value in lora_eval_results.items():
    print(f"  {key}: {value}")

print("\nIA3 Evaluation Results:")
for key, value in ia3_eval_results.items():
    print(f"  {key}: {value}")

LoRA eval_loss: 11.256444931030273
IA3 eval_loss: 11.256444931030273

--- Detailed Comparison ---
LoRA Evaluation Results:
  eval_loss: 11.256444931030273
  eval_runtime: 2.0741
  eval_samples_per_second: 9.643
  eval_steps_per_second: 2.411
  epoch: 3.0

IA3 Evaluation Results:
  eval_loss: 11.256444931030273
  eval_runtime: 2.0348
  eval_samples_per_second: 9.829
  eval_steps_per_second: 2.457
  epoch: 3.0
